# Raspberry Pi 4
> [Main Table of Contents](../README.md)

## In This Notebook
> References: [Raspberry Pi Docs](https://www.raspberrypi.com/documentation/computers/remote-access.html#setting-up-an-ssh-server)  
- Use `apt` as package manager
    - Useful commands
- Enable Raspberry Pi OS SSH
- SSH headless entry into Raspberry Pi OS

## Use `apt` as package manager

Command | Description
--- | ---
`apt update`| Update all installed packages 
`dpkg --get-selections` | Get list of installed packages
`apt list --installed` | Get *detailed* list of installed packages


## Enable Raspberry Pi OS SSH
- Rasp Pi Menu dropdown -> Preferences -> Raspberry Pi Configuration -> Interfaces -> Toggle SSH

## SSH headless entry into Raspberry Pi OS using password (passwordless using keys is not setup)
- Get Rasp Pi IP address
    ```python
    # From Rasp Pi terminal
    hostname -I

    # From another computer on same network
    ping raspberrypi.local    # This is return rasp pi's ip address

    ```
- Terminal on another device
    - Username & password is that for Raspberry Pi and configured when first set up the Raspberry Pi OS
        ```
        ssh <username>@<rasp pi ip>
        ```
